Scraping inspirations for finn.no:
-[python](https://github.com/qiangwennorge/ScrapeFinnBolig)
-[node.js-scraper](https://github.com/Lekesoldat/finn-scraper)

Example of bussiness property is [this building](https://www.finn.no/realestate/businesssale/ad.html?finnkode=335466271). 
It can be scraped using (BeautifulSoup)[https://www.crummy.com/software/BeautifulSoup/bs4/doc/].

In [57]:
import requests
from bs4 import BeautifulSoup
# for splitting strings
import re

For now we want to focus on Sarpsborg. By adding a paramter to the URL, we can narrow our search for the listings in Sarpsborg

In [9]:
mainurl = 'https://www.finn.no/realestate/businesssale/search.html?location='
# Sarpsborg location
sarpsborg = '1.20002.20023'

In [34]:
def RequestAndScrape(url) :
    r = requests.get(url)
    if r.status_code == 200 :
        soup = BeautifulSoup(r.content, 'html.parser')
        return soup
    else :
        print('Bad Request:', r.status_code)

In [38]:
def FetchListings(soup) :
    arr_listings = []
    # find the listings in articles with the following class
    listings = soup.find_all('a', {'class':'sf-search-ad-link'} )
    i = 0
    for listing in listings : 
        # get the value of the url
        listing_url = listing['href']
        arr_listings.append(listing_url)
        i += 1
    # count listings
    print(i)
    return arr_listings

In [41]:
url = mainurl + sarpsborg
soup = RequestAndScrape(url)

listings = FetchListings(soup)


20


Per listing, I want the following information: 
    - Title of listing
    - Adress of listing
    - Type of listing
    - Area information
    - Real Estate Agent

In [65]:
for listing in listings : 
    print(listing)
    listing_data = RequestAndScrape(listing)
    
    title = listing_data.find('h1').text
    # adress is streetname number, postcode city
    list_adress = listing_data.find('span', {'data-testid':'object-address'}).text
    print(list_adress)


https://www.finn.no/realestate/businesssale/ad.html?finnkode=337701162
Kirkegata 29, 1721 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=335466271
Glomveien 6, 1710 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=334626589
Vestbyveien 1 A-I, 1734 Hafslundsøy
https://www.finn.no/realestate/businesssale/ad.html?finnkode=309991654
Roald Amundsens gate 19 og 21, 1723 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=333528680
Storgata 31, 1723 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=333453326
Bjørnsons gate 6/8, 1724 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=332661701
St. Marie gate 2, 1706 Sarpsborg
https://www.finn.no/realestate/businesssale/ad.html?finnkode=329947820
Kampenesmosen - Kombinasjonsbygg, 1739 Borgenhaugen
https://www.finn.no/realestate/businesssale/ad.html?finnkode=329929465
Alvimveien 59 - Lagerseksjoner, 1722 Sarpsborg
https://www.finn.no/realest

In [68]:
# geocode adress
adress = re.split(', | ', list_adress )
print(adress)
street  = adress[0] + adress[1]
postalcode = adress[2]
city = adress[3]

geocode_url = 'https://nominatim.openstreetmap.org/search'
params = dict (
    adressdetails = '1',
    extratags = '1',
    namedetails = '1',
    limit = '1',
    polygon_geojson= '1',
    format = 'geojson',
    street = street,
    city = city,
    postalcode = postalcode
)

r = requests.get(geocode_url, params=params)
features = r.json()['features']

['Løenveien', '1', '1747', 'Skjeberg']
